In [2]:
#import pacakges 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import math
import numpy as np
import cv2

# 1) Use ginput to find correspondances

In [40]:
def getPoints(image):
    img1 = Image.open(image) 
    plt.figure(1)
    plt.imshow(img1)
    print("Please click") 
    x = plt.ginput(4)
    return x

In [41]:
print ("points x ", getPoints('./Images/image1.jpg'))
print("points y", getPoints('./Images/image2.jpg'))

Please click


/usr/lib/python3/dist-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


points x  [(817.0483870967741, 479.3225806451613), (482.5967741935484, 359.58064516129036), (245.1774193548387, 270.8064516129033), (600.2741935483871, 200.61290322580646)]
Please click
points y [(668.4032258064516, 456.61290322580646), (457.82258064516134, 376.0967741935484), (276.14516129032256, 291.45161290322585), (416.5322580645161, 578.4193548387096)]


#

In [46]:
#get x,y,x',y' arrays from cores
def getArrays(src,dest):
    x=[]
    y=[]
    xdash=[]
    ydash=[]

    for i in range(4):
        x.append(src[i][0])
        y.append(src[i][1])
        xdash.append(dest[i][0])
        ydash.append(dest[i][1])

    return x,y,xdash,ydash

In [47]:
#homography parameters implemntation
def findHomography (src, dest):
    b = dest.flatten().reshape(-1,1)

    x,y,xdash,ydash = getArrays(src,dest)

    M = np.array([
          [x[0],y[0],1,0,0,0,-x[0]*xdash[0], -y[0]*xdash[0]],
          [0,0,0,x[0],y[0],1,-x[0]*ydash[0], -y[0]*ydash[0]],
          [x[1],y[1],1,0,0,0,-x[1]*xdash[1],-y[1]*xdash[1]],
          [0,0,0,x[1],y[1],1,-x[1]*ydash[1],-y[1]*ydash[1]],
          [x[2],y[2],1,0,0,0,-x[2]*xdash[2],-y[2]*xdash[2]],
          [0,0,0,x[2],y[2],1,-x[2]*ydash[2],-y[2]*ydash[2]],
          [x[3],y[3],1,0,0,0,-x[3]*xdash[3],-y[3]*xdash[3]],
          [0,0,0,x[3],y[3],1,-x[3]*ydash[3],-y[3]*ydash[3]]
      ])
    a = np.dot(np.linalg.inv(M),b)
    a = np.append(a,1)
    return a.reshape(3,3)

In [48]:
#test to check homography function
pts_src = np.array([[141, 131], [480, 159], [493, 630],[64, 601]])
pts_dst = np.array([[318, 256],[534, 372],[316, 670],[73, 473]])
h=findHomography(pts_src,pts_dst)
print("calculated homography", h)
h, status = cv2.findHomography(pts_src, pts_dst)
print("actual homogrpahy", h)


calculated homography [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]
actual homogrpahy [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]
